# CODE SUPPORT

### Imports and settings

In [1457]:
# Data extraction
from ucimlrepo import fetch_ucirepo

# Data handling
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import pickle

# Data viz
import warnings
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

# ML packages
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [7]:
warnings.filterwarnings('ignore')

### Helper Functions

In [459]:
def uci_heart_data():
    '''
        Downloads dataset from the UCI Repository: Heart Disease.

        Parameters:
            None.
        
        Returns:
            DataFrame: heart disease dataset.
    '''
    heart_disease = fetch_ucirepo(name='Heart Disease')

    df_features = heart_disease.data.features
    df_target = heart_disease.data.targets
    df_full = pd.concat([df_features, df_target], axis=1)
    
    return df_full


def recall_f1_cv(x_cv, y_cv, model_name, model, kfold=5, verbose=False):
    '''
    Cross-Validation for classifier models.

    This CV helper function is made for non-ordinal classifier models,
    thinking about the redution of False Negatives in our results.
    Scores: Recall and F1.
    '''
    recall_list = []
    f1_list = []

    # We are going to use an index reset to apply our kfold limits.
    df_aux = pd.concat([x_cv, y_cv], axis=1)
    df_aux.reset_index(inplace=True, drop=True)
    df_cv = df_aux.reset_index()

    index_max = df_cv['index'].max()

    for k in range( 1, (kfold + 1) ):
        # This divides the data in kfold parts.
        if verbose:
            # For verbose=True, shows the KFold input.
            print( '\nKFold Number: {} / {}'.format(k, kfold) )
        
        # Parameter to include last index in the split.
        z = 0
        if k == kfold:
            z = 1
            
        # Splitting train-validation.
        df_train = df_cv[
            (df_cv['index'] < (index_max * (k-1) / kfold)) | 
            (df_cv['index'] >= (index_max * (k+z) / kfold))]
        if verbose:
            print(df_train.shape)
        
        df_val = df_cv[
            (df_cv['index'] >= (index_max * (k-1) / kfold)) & 
            (df_cv['index'] < (index_max * (k+z) / kfold))]
        if verbose:
            print(df_val.shape)

        # Training dataset.
        x_train = df_train.drop(['index', 'num'], axis=1)
        y_train = df_train['num']

        # Validation dataset.
        x_val = df_val.drop(['index', 'num'], axis=1)
        y_val = df_val['num']

        # Model training.
        m = model.fit( x_train, y_train )

        # Prediction.
        y_pred = m.predict(x_val)

        # Storing Recall and F1 scores.
        recall_list.append(recall_score(y_val, y_pred))
        f1_list.append(f1_score(y_val, y_pred))

    # Performance results
    return pd.DataFrame({'Model Name': model_name,
                        'Recall Score': np.round(np.mean(recall_list), 2).astype( str ),
                        'F1 Score': np.round(np.mean(f1_list), 2).astype( str )},
                        index=[0])

# 0 BUSINESS UNDERSTANDING

### CRISP for Data Science

This is the methodology I use in my projects.

In [ ]:
# Loading crisp img
img = Image.open('img/crisp.png')

# Show
display(img)

### Understanding the Problem

Then we begin tackling the problem by understanding it.

Objective of this study:
    - Supporting the development of a heart disease prevention system!

My job as a Data Scientist is to create a predictive model to identify patients more likely to develop heart diseases.


Dataset available in the URL from UC Irvine (UCI) ML Repository: https://archive.ics.uci.edu/ml/datasets/heart+disease.
It's made of 4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach.

### Understanding the Data

As I don't have support from a medical team, my decisions are solely made out of my dataset limited interpretation.

Below, we can see a table with the variables information summary, taken directly from the UCI ML Repository.

In [ ]:
# Loading data_features img
img = Image.open('img/dataset_variables.png')

# Show
display(img)

The Target feature also have this extra info:

num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)

Because we have the Target feature, it's possible to use supervised models to predict whether or not new patients are prone to heart diseases.

The next steps are Data Extraction and Data Cleaning. We can better understand the data he have by diving into it.

In [ ]:
# Loading timeline img
img = Image.open('img/timeline.png')

# Show
display(img)

# 1 DATA EXTRACTION

In [12]:
# Downloading the dataset from UCI.
df1 = uci_heart_data()

# 2 DATA CLEANING

In [716]:
# Now we begin to understand how the dataset was built in more depth.
df2 = df1.copy()

### 2.1 Data Description

In [ ]:
# Just taking a look at the dataframe.
df2.head(3)

In [ ]:
# First let's get rid of duplications, if they exist.
df2.drop_duplicates()

# Checking dataframe size.
print( 'Rows: {}'.format( df2.shape[0] ) )
print( 'Cols: {}'.format( df2.shape[1] ) )

In [ ]:
# Displaying information about variables types and missing values.
type_value_analysis = pd.concat(
    [df2.dtypes, df2.isna().sum()], 
    axis=1, 
    keys=['Data Types', 'Missing Values'])
print(type_value_analysis)

### 2.2 Missing Values

In [ ]:
# Trying to understand the missing values.
df2[(df2['ca'].isna()) | (df2['thal'].isna())]

In [721]:
# I don't have any other information that could help determine the missing values 
# for 'ca' (flouroscopy result) or 'thal' (whatever this is). 
# 
# The decision here is to drop them. It's just 2.3% of our dataset, we'll survive.
df2 = df2.dropna()

### 2.3 Data Types

In [ ]:
# We have to take a look in the variables with data type float64, as the 
# int64 alreary have their values determined by the nature of its type.
# 
# These are 'oldpeak', 'ca' and 'thal'. Let's check their values. 
# 
# P.S.: UCI Repository says that 'oldpeak' and 'ca' are integer and 'thal' 
# is categorical.
unique_values_dict = {
    'oldpeak': list(df2['oldpeak'].unique()),
    'ca': list(df2['ca'].unique()),
    'thal': list(df2['thal'].unique())
}

for i in unique_values_dict:
    print('{}: {}'.format(i, unique_values_dict[i]))

In [723]:
# As we can see, 'ca' and 'thal' can be stored as integer. But, contrary to 
# the UCI recommendation, 'oldpeak' is indeed float.
variables_to_integer = ['ca', 'thal']

for i in variables_to_integer:
    df2[i] = df2[i].astype('int64')

### 2.4 Identifying Categorical and Numerical Data

In [ ]:
# Let's display unique values for each variable. Then we can decide on
# the nature of the data attributes.
attributes = list(df2.columns)
attributes_dict = {}

for att in attributes:
    attributes_dict[att] = list(df2[att].unique())

for i, j in attributes_dict.items():
    print('{}: {}'.format(i, j))

In [725]:
# The values for 'num' are not as expected from the data description from UCI.
# As we should have only 0 or 1, for False and True.

In [726]:
# Complete list of attributes:
attributes = list(df2.columns)

# For the numerical attributes, we can identify:
num_attributes = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# And for the categorical ones, we have:
cat_attributes = [att for att in attributes if att not in num_attributes]

### 2.5 Adjusting Values

In [727]:
# As we saw, 'num' values are not as expected. We can follow 
# with 0 as 0, but 1, 2, 3, 4 as 1. 
# If I'd take a guess, it may identify from which of the four
# databases these positives come from.
df2.loc[df2['num'] > 0, 'num'] = 1

### 2.6 Identifying Outliers

In [ ]:
# Plotting histogram to identify weird patterns.
df2[num_attributes].hist(bins=20, figsize=(10, 8));

In [ ]:
# Plotting histogram to identify weird patterns.
df2[cat_attributes].hist(bins=20, figsize=(10, 8));

In [730]:
# Everything looks fine

# 3 EXPLORATORY DATA ANALYSIS

In [736]:
# It's adventure time.
df3 = df2.copy()

### 3.1 Hypothesis Creation 

In [737]:
# NUMERICAL ATTRIBUTES
# 1. age - Risk patients tend to be older;

# 2. trestbps - Risk patients are more common to have higher resting
# blood pressure;

# 3. chol - It's more common to find high serum cholesterol patients
# among the risk group;

# 4. thalach - maximum heart rate achieved higher for risk patients,
# athletes tend to have lower heart rate overall;

# 5. oldpeak - ST segment instability are common in patients with heart
# disease.


# CATEGORICAL ATTRIBUTES
# 6. sex - Women (0) take more care of themselves and see doctors more 
# often, meaning lower risk of developing diseases in general;

# 7. cp - chest pain (1, 2, 3) may indicate problems around the chest, 
# anginal (1, 2) often indicate a heart disease (value 4 should be more 
# common in healthy patients);

# 8. fbs - high sugar (1) is usually related to heart diseases;

# 9. restecg - abnormal (1) and compromised (2) ECG results should be
# more often on risk patients;

# 10. exang - angina by exercising (1) should indicate risk;

# 11. slope - ST slope should be flat (2), most important cause of
# ST segment abnormality is myocardial ischaemia or infarction 
# (source: litfl.com/st-segment-ecg-library/ accessed in 2024-sep-07);

# 12. ca - I don't know what result we should get, let's assume
# that lower vessels mean higher risk;

# 13. thal - normal (3) means lower risk, defects (6, 7) should be 
# more common in risk patients.


### 3.2 Descriptive Analysis for each Variable

#### 3.2.1 Numerical Attributes

##### <font color='lightblue'> Univariate Analysis </font>

In [ ]:
# Central Tendency
ct1 = pd.DataFrame( df3[num_attributes].apply( np.mean ) ).T
ct2 = pd.DataFrame( df3[num_attributes].apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( df3[num_attributes].apply( np.std ) ).T
d2 = pd.DataFrame( df3[num_attributes].apply( min ) ).T
d3 = pd.DataFrame( df3[num_attributes].apply( max ) ).T
d4 = pd.DataFrame( df3[num_attributes].apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( df3[num_attributes].apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( df3[num_attributes].apply( lambda x: x.kurtosis() ) ).T

summary = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
summary.columns = [ 'attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis' ]

summary

In [739]:
# In general, we have distributions close to Normal, with attention
# to 'chol' with high kurtosis, indicating a high peak around the mean value.
# And 'oldpeak' with high positive skewness, showing its tendency to lower
# values, and high kurtosis, also highlighting a great peak.

In [ ]:
df3[num_attributes].hist( bins=80 );

In [741]:
# We can see an accumulation of 'oldpeak' in point zero and
# this can be investigated further.

##### <font color='lightblue'> Bivariate Analysis </font>

In [ ]:
# 'oldpeak' analysis just to get a hang of it.
# Figure size.
plt.figure(figsize=(14, 6))

# Plot for healthy patients.
sns.kdeplot(df3.loc[df3['num'] == 0, 'oldpeak'], label='Healthy', color='dodgerblue', shade=True)

# Plot for risk patients.
sns.kdeplot(df3.loc[df3['num'] == 1, 'oldpeak'], label='Risk', color='darkred', shade=True)

# Show legend and graph.
plt.legend()
plt.show()

In [ ]:
# We can see that 'oldpeak' serves as a good indicator for heart health.
# From that, we can generalize and see how the other variables behave too.
for att in num_attributes:
    # Figure settings.
    plt.figure(figsize=(8, 3))

    # Plots for healthy and risk patients
    sns.kdeplot(df3.loc[df3['num'] == 0, att], label='Healthy', bw_adjust=0.5, color='dodgerblue', shade=True)
    sns.kdeplot(df3.loc[df3['num'] == 1, att], label='Risk', bw_adjust=0.5, color='darkred', shade=True)

    # Show plots.
    plt.legend()
    plt.show()

In [744]:
# NUMERICAL ATTRIBUTES
# 1. age - Risk patients tend to be older;
#   * TRUE *

# 2. trestbps - Risk patients are more common to have higher resting
# blood pressure;
#   * TRUE * But the difference between groups is small.

# 3. chol - It's more common to find high serum cholesterol patients
# among the risk group;
#   * TRUE *

# 4. thalach - maximum heart rate achieved higher for risk patients,
# athletes tend to have lower heart rate overall;
#   * FALSE * Looks like healthy people has some extra power.

# 5. oldpeak - ST segment instability are common in patients with heart
# disease.
#   * TRUE * Very significant difference between groups.

#### 3.2.2 Categorical Attributes

##### <font color='lightpink'> Uni and Bivariate Analysis </font>

In [ ]:
for att in cat_attributes:
    # Target variable plot.
    if att == 'num':
        fig, axes = plt.subplots(1, 1, figsize=(6, 3))
        sns.countplot(x=att, data=df3)

        # Show plot
        plt.tight_layout()
        plt.show()

        # Go to next att.
        continue

    # Categorical Features plot.
    # Figures settings.
    fig, axes = plt.subplots(1, 2, figsize=(12, 3))

    # Count of the attribute.
    sns.countplot(x=att, data=df3, ax=axes[0])

    # Comparison to the target variable.
    sns.kdeplot(df3.loc[df3['num'] == 0, att], label='Healthy', color='dodgerblue', shade=True, bw_adjust=0.1, ax=axes[1])
    sns.kdeplot(df3.loc[df3['num'] == 1, att], label='Risk', color='darkred', shade=True, bw_adjust=0.1, ax=axes[1])

    # Show plots
    plt.legend()
    plt.tight_layout()
    plt.show()

In [746]:
# CATEGORICAL ATTRIBUTES
# 6. sex - Women (0) take more care of themselves and see doctors more 
# often, meaning lower risk of developing diseases in general;
#   * TRUE *

# 7. cp - chest pain (1, 2, 3) may indicate problems around the chest, 
# anginal (1, 2) often indicate a heart disease (value 4 should be more 
# common in healthy patients);
#   * FALSE * eXCUSE ME, WTf? Just found out that ischemia can be 
# * "silent"... Scary.

# 8. fbs - high sugar (1) is usually related to heart diseases;
#   * FALSE * Little to no difference between groups.

# 9. restecg - abnormal (1) and compromised (2) ECG results should be
# more often on risk patients;
#   * TRUE *

# 10. exang - angina by exercising (1) should indicate risk;
#   * TRUE *

# 11. slope - ST slope should be flat (2), most important cause of
# ST segment abnormality is myocardial ischaemia or infarction 
# (source: litfl.com/st-segment-ecg-library/ accessed in 2024-sep-05);
#   * FALSE * Apparently, the expected slope while exercising is 
# * different from rest slope. Up slope is the healthy one.

# 12. ca - I don't know what result we should get, let's assume
# that less color vessels mean higher risk;
#   * FALSE * 0 is healthy, the others indicate risk. Further
# * research let me understand that are multiple types of color-
# * coding, depending on the purpose of the Angiography. We should
# * get better info with the medical team about the procedure.

# 13. thal - normal (3) means lower risk, defects (6, 7) should be 
# more common in risk patients.
#   * TRUE *


# Also, we have a lot of risk patients, almost half of our sample (> 50% diameter narrowing).

### 3.3 Multivariate Analysis

In [ ]:
# Now we can try to find other correlations between variables
# and see how much each one impact the target variable compared
# to one another.
#
# Seaborn pairplot in histograms can give us a good 2-D overview
# of our numerical attributes related to target.
pplot_attributes = num_attributes.copy()
pplot_attributes.append('num')
sns.pairplot(df3[pplot_attributes], hue = 'num', diag_kind='hist', plot_kws={'alpha':0.5});

In [748]:
# As an 2-dimensional hitplot, being the target variable indicated
# by color. We are looking for diagonal shifts between the two
# color spreads.
#
# Diagonal spreads mean greater correlation betwen two features,
# and diagonal shifts mean they are both greatly affecting the target.
#
# Most diagonal-ish spreads:
# age x thalach, age x trestbps, trestbps x thalach, 
# thalach x oldpeak.
#
# Most evident diagonal shifts:
# age x thalach, age x oldpeak, thalach x oldpeak
#
# So we should give double attention to 'age', 'thalach' and 'oldpeak'.

In [ ]:
# Examining a heatmap, we could confirm the assumptions above
# and evaluate how the categorical attributes play along.
#
# Figures layout.
fig,ax = plt.subplots(figsize=(15,8))

# Getting the correlation matrix.
corr = df3.corr()
corr = corr.round(4)

# Taking the upper triangle of the correlation matrix.
matrix = np.triu(corr)

# Using the upper triangle matrix as mask to only show what's left.
sns.heatmap(corr, center=0, cmap='RdBu', annot=True, mask=matrix)

In [750]:
# Here we can confirm that 'thalach' and 'oldpeak' have
# the greatest influence over 'num', followed by 'age',
# when watching only num_attributes.
#
# For cat_attributes, we can highlight 'cp', 'ca', 'exang'
#  and 'thal', followed by 'sex' and 'slope'.
#
# Another interesting result is 'fbs' showing little to
# no correlation to every other variable, except for
# 'age', 'trestbps' and 'ca'. It may affect our target
# indirectly but we should try taking it off our future
# models to investigate its impact.


In [ ]:
# Further investigating the case of asymptomatic chest 
# pain, I found good information about how it's sex related
# even though it looks to have no correlation according
# to the heatmap above. 
# Now looking at how it differs from men to women.
#
# Plotting relative distribution of 'cp' for men and women,
# healthy and in risk.
# Figures settings
fig, axes = plt.subplots(1, 2, figsize=(12, 3))

def plot_kde_normalized(ax, data, label, color):
    # Compute KDE
    kde = gaussian_kde(data, bw_method=0.03)
    x = np.linspace(1, 4, 1000)
    y = kde(x)
    
    # Normalize to 100%
    y = y / y.sum()
    
    # Plot
    ax.plot(x, y, label=label, color=color)
    ax.fill_between(x, y, color=color, alpha=0.2)

# Women
plot_kde_normalized(axes[0], df3.loc[(df3['num'] == 0) & (df3['sex'] == 0), 'cp'], label='Healthy', color='dodgerblue')
plot_kde_normalized(axes[0], df3.loc[(df3['num'] == 1) & (df3['sex'] == 0), 'cp'], label='Risk', color='darkred')

# Men
plot_kde_normalized(axes[1], df3.loc[(df3['num'] == 0) & (df3['sex'] == 1), 'cp'], label='Healthy', color='dodgerblue')
plot_kde_normalized(axes[1], df3.loc[(df3['num'] == 1) & (df3['sex'] == 1), 'cp'], label='Risk', color='darkred')


# Adjusting layout, titles and legends
axes[0].set_title('Women')
axes[0].set_ylim(0, 0.11)
axes[1].set_title('Men')
axes[1].set_ylim(0, 0.11)

axes[0].legend()
axes[1].legend()

plt.tight_layout()
plt.show()


In [752]:
# Typical Angina: Great indicator for men in risk, it's not a symptom in women;
# Atypical Angina: Great indicator for women in risk, less frequent in men;
# Non-Anginal Chest Pain: Great indicator for men, not for women;
# Asymptommatic: Way more common for women.
#
# With further research, I could find that the diagnosis for men and women are 
# quite different. Being chest pain, commonly spread as a must have sympton, 
# being greatly relevant for diagnosing men, but not women.
#
# According to the Hospital Einstein blog (https://vidasaudavel.einstein.br/infarto
# -em-mulheres-conheca-mitos-e-verdades-sobre-ataque-cardiaco/, accessed in 2024-sep
# -06):
# "[...] women describe chest pain as pressure or a tight feeling, and not as an 
# accute pain. Other frequent symptoms in female sex patients are:
# Burning skin; pain in neck, shoulders, face and jaw; difficulty breathing; 
# fatigue; palpitations."
#
# From the National Library of Medicine (https://www.ncbi.nlm.nih.gov/medgen/149267#:
# ~:text=Definition,from%20NCI%5D, accessed in 2024-sep-06), they define Atypical
# Angina as:
# "Angina pectoris which does not have associated classical symptoms of chest pain. 
# Symptoms may include weakness, nausea, or sweating."


In [753]:
# That show that we can not rely solely on a heatmap
# to determine feature correlation. Specially because 
# the value 4 of 'cp' has huge impact on the distribution.


### 3.4 Feature Engineering

In [754]:
# Best correlation found until now was between
# symptomns and sex, and I have a plan for it when
# it's time to encode the cat_attributes.

### 3.5 Removing Outliers

In [ ]:
df3.plot(kind='box', subplots=True, layout=(2, 7), figsize=(20,8));

In [ ]:
# We can observe that 'trestbps', 'chol' and 'odpeak' show
# outliers. We can make a for loop to get rid of those cases.
df3_filtered = df3.copy()

for att in num_attributes:
    # Getting Q1 and Q3.
    q1 = np.percentile(df3_filtered[att], 25)
    q3 = np.percentile(df3_filtered[att], 75)

    # Then the Interquartile Range (IQR).
    iqr = q3 - q1

    # And determine Inferior and Superior Limits.
    # Everything out of these boundaries are gonna be treated
    # as outliers.
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    df3_filtered = df3_filtered[(df3_filtered[att] >= lower_bound) & 
                                (df3_filtered[att] <= upper_bound)]
    
print(f'Original df3 shape: {df3.shape}')
print(f'Filtered df3 shape: {df3_filtered.shape}')


In [757]:
# With outliers removed in our df3_filtered, let's
# copy it to our df3.
df3 = df3_filtered.copy()

# 4 DATA MODELING

In [1109]:
# Time to prepare our data for the ML algorithms.
df4 = df3.copy()

### 4.1 Data Preprocessing

#### 4.1.1 Scalers

In [1110]:
# As I'm planning to use tree based models, I was not
# going to scale our numerical attributes. But they always 
# work better when doing it for some reason.
# Not much better, but it helps.
#
# MinMaxScaler for old_peak, because this one does not have a good
# bell shape. And StandardScaler for the rest of the num attributes.
cols_minmax = ['oldpeak']
cols_std = num_attributes = [att for att in num_attributes if att != 'oldpeak']

df4[cols_minmax] = MinMaxScaler().fit_transform(df4[cols_minmax])
df4[cols_std] = StandardScaler().fit_transform(df4[cols_std])

#### 4.1.2 Encoders

In [ ]:
for att in cat_attributes:
    print('{}, values: {}'.format(att, df3[att].unique()))

In [1112]:
# sex: fine (labeled);
# * cp: It's already label encoded, but I'll reduce the amount of labels;
# fbs: fine (labeled);
# restecg: fine (ordinal);
# exang: fine (labeled);
# slope: fine (ordinal);
# ca: fine (ordinal);
# * thal: encoding reordination;
# num: target, fine.

In [ ]:
# The'cp' values 1 and 3 are almost irrelevant for women.
# On the other hand, they are good indicators of heart 
# disease for men.
#
# Value 2 is way more influent on women then on men.
#
# As a solution, we can separate male from female related 
# symptoms, as they are really dependent on sex. This will
# help the ML algorithms work less.
#
# New Chest Pain 'cp':
# 0 = female related symptoms[former 2]),
# 1 = male related symptoms [former 1 and 3],
# 2 = asymptomatic [former 4].
if 4 in list(df3['cp'].unique()):
    df3['cp'] = df3['cp'].replace(to_replace=[1, 2, 3, 4], value=[1, 0, 1, 2])
else:
    print('Already adjusted.')

# 'thal' values only need a reordination.
if 7 in list(df3['thal'].unique()):
    df3['thal'] = df3['thal'].replace(to_replace=[3, 6, 7], value=[0, 1, 2])
else:
    print('Already adjusted.')

### 4.2 Training-Validation Split

In [ ]:
# Firts, let's define our X, y data.
X = df4.drop(columns=['num'])
y = df4['num']

# Input (X) and target (y) selection
print('Data shape - X: {}, y: {}'.format(X.shape, y.shape))

In [ ]:
# In my planning I was going to write my own frunction for
# this, but why?
# From the scikit-learn documentation, we can use the
# train_test_split helper function to split our data and
# save a replicable state.
#
# Replicable state number 11,
# Test sample of 20%.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=11)

print('Training data sample - X: {}, y: {}'.format(X_train.shape, y_train.shape))
print('Testing data sample - X: {}, y: {}'.format(X_test.shape, y_test.shape))

### 4.3 Cross-Validation

#### 4.3.1 Decision Tree Classifier

In [ ]:
# Our cross-validation will divide our training data
# in 4 groups, to make it close to the size of our test
# sample.
#
# First we run our baseline DecisionTreeClassifier.
def dt_result_cv():
    model = DecisionTreeClassifier()
    dt_result_cv = recall_f1_cv(x_cv=X_train, y_cv=y_train, kfold=5, 
                                model_name='Decision Tree', model=model, verbose=False)

    return dt_result_cv

dt_result_cv()

#### 4.3.2 Random Forest

In [ ]:
# Baseline Random Forest
def rf_result_cv():
    model = RandomForestClassifier()
    rf_result_cv = recall_f1_cv(x_cv=X_train, y_cv=y_train, kfold=5, 
                                model_name='Random Forest', model=model, verbose=False)

    return rf_result_cv

rf_result_cv()

#### 4.3.3 XGboost

In [ ]:
# Baseline XGBoost
model = XGBClassifier()
xgb_result_cv = recall_f1_cv(x_cv=X_train, y_cv=y_train, kfold=8, 
                             model_name='XGBoost', model=model, verbose=False)

xgb_result_cv

In [ ]:
model = XGBClassifier(
    random_state=None,
    objective='binary:logistic',
    n_estimators=900,
    eta=0.01,
    max_depth=100,
    min_samples_leaf=5,
    min_child_weight=0,
    colsample_bytree= 0.5,
    subsample=0.5
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

### 4.4 Choosing Model

In [ ]:
result = dt_result_cv()
result.dtypes

In [ ]:
# XGBoost is giving the same result everytime I run it.
# So it's not gonna be used.
#
# Models that we can hypertune: DecisionTree and RandomForest.
'''
possible_models = [dt_result_cv(), rf_result_cv()]

# Let's see how they perform on average for baseline.
for m in possible_models:

    model_result = pd.DataFrame(columns=['Model Name', 'Recall Score', 'F1 Score'])

    for i in range(0, 50000):
        model_result = pd.concat([model_result, m])

    avg_recall = model_result['Recall Score'].astype(float).mean()
    avg_recall = round(avg_recall, 2)
    avg_f1 = model_result['F1 Score'].astype(float).mean()
    avg_f1 = round(avg_f1, 2)

    m_name = m.loc[0, 'Model Name']
    print('Model Name: {}'.format(m_name))
    print('Average Recall: {}\nAverage F1: {}\n'.format(avg_recall, avg_f1))
'''
# Running the comparison, we see that both have similar performance
# on Recall, but Random Forest wins on F1.
# Then we go with Random Forest.

# 5 HYPERPARAMETER FINE TUNING

In [1272]:
# From now on, we are gonna use Random Forest for fine tuning.
df5 = df4.copy()

### 5.1 Grid Searching best Parameters

In [ ]:
# Defining model.
rf = RandomForestClassifier()

# Let's define the parameters for our grid search.
param_grid = {
    'n_estimators': range(10, 51),
    'max_depth' : range(2, 11),
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_impurity_decrease': [0.1, 0.01]
}

# Setting GridSearchCV up.
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='recall',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

print('Best parameters:')
print(grid_search.best_params_)

In [ ]:
rf_tuned = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                  max_depth=best_params['max_depth'],
                                  max_features=best_params['max_features'],
                                  random_state=11)

# Train and predict.
rf_tuned.fit(X_train, y_train)
y_pred = rf_tuned.predict(X_test)

# Show results on Recall and F1 scores.
recall = round(recall_score(y_test, y_pred), 2)
f1 = round(f1_score(y_test, y_pred), 2)
print('Recall Score: {}'.format(recall))
print('F1 Score: {}'.format(f1))

### 5.2 Saving our Model

In [1460]:
pickle.dump( rf_tuned, open('model/heart_disease_tuned_rf.pkl', 'wb') )

# 6 MODEL EVALUATION